In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence


In [2]:
from src.extractors.utils import extract_images_and_tables_as_images, extract_images

In [3]:
PATH = "C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-document-intelligence\\notebooks\\dev\\instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf"

In [4]:
from IPython.display import display
from PIL import Image

In [14]:
# If your image is in a different directory, provide the full path
image_path = r"C:\path\to\your\image\instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788_image_page_15_1.png"

display(Image(image_path))

TypeError: 'module' object is not callable

In [9]:
Path_File = "C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-document-intelligence\\notebooks\\dev\\instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf"
output_path = "C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-document-intelligence\\notebooks\\dev\\output\\"

In [11]:
import pdfplumber

In [24]:
pdf = pdfplumber.open(Path_File)
page = pdf.pages[4]
a = page.extract_table()

In [27]:
import pdfminer
from pdfminer.image import ImageWriter
from pdfminer.high_level import extract_pages

pages = list(extract_pages(Path_File))
page = pages[0]


def get_image(layout_object):
    if isinstance(layout_object, pdfminer.layout.LTImage):
        return layout_object
    if isinstance(layout_object, pdfminer.layout.LTContainer):
        for child in layout_object:
            return get_image(child)
    else:
        return None


def save_images_from_page(page: pdfminer.layout.LTPage):
    images = list(filter(bool, map(get_image, page)))
    iw = ImageWriter(output_path)
    for image in images:
        iw.export_image(image)


save_images_from_page(page)

In [49]:
from pypdf import PdfReader

reader = PdfReader(Path_File)

page = reader.pages[1]

In [51]:
import fitz
import io
from PIL import Image

# file path you want to extract images from
file = Path_File

In [52]:
# open the file
pdf_file = fitz.open(file)
# iterate over PDF pages
for page_index in range(pdf_file.page_count):
    # get the page itself
    page = pdf_file[page_index]
    image_li = page.get_images()
    # printing number of images found in this page
    # page index starts from 0 hence adding 1 to its content
    if image_li:
        print(f"[+] Found a total of {len(image_li)} images in page {page_index+1}")
    else:
        print(f"[!] No images found on page {page_index+1}")
    for image_index, img in enumerate(page.get_images(), start=1):
        # get the XREF of the image
        xref = img[0]
        # extract the image bytes
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]
        # get the image extension
        image_ext = base_image["ext"]
        # load it to PIL
        image = Image.open(io.BytesIO(image_bytes))
        # save it to local disk
        image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))

[+] Found a total of 3 images in page 1
[!] No images found on page 2
[+] Found a total of 1 images in page 3
[+] Found a total of 1 images in page 4
[!] No images found on page 5
[+] Found a total of 1 images in page 6
[+] Found a total of 1 images in page 7
[+] Found a total of 2 images in page 8


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000255666A6A90>

In [50]:
page.images

In [46]:
from pypdf import PdfReader

reader = PdfReader(Path_File)

page = reader.pages[0]
count = 0

for image_file_object in page.images:
    with open(str(count) + image_file_object.name, "wb") as fp:
        fp.write(image_file_object.data)
        count += 1

In [28]:
from pypdf import PdfReader

In [42]:
import PyPDF2
import io

In [44]:
import os


def extract_images_from_pdf(pdf_path, output_path):
    pdf_file = open(pdf_path, "rb")
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # If the output path does not exist, create it
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Iterate through each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]

        if "/XObject" in page["/Resources"]:
            xObject = page["/Resources"]["/XObject"].get_object()

            for obj in xObject:
                if xObject[obj]["/Subtype"] == "/Image":
                    try:
                        # Identify image format
                        if xObject[obj]["/Filter"] == "/DCTDecode":
                            file_ext = "jpg"
                        elif xObject[obj]["/Filter"] == "/JPXDecode":
                            file_ext = "jp2"
                        else:
                            file_ext = "png"

                        data = xObject[obj].get_images().get_data()
                        image = Image.open(io.BytesIO(data))
                        image_filename = os.path.join(
                            output_path, f"page{page_num+1}_{obj[1:]}.{file_ext}"
                        )
                        image.save(image_filename)
                        print(f"Saved image from page {page_num+1}: {image_filename}")
                    except Exception as e:
                        print(f"Error on page {page_num+1}: {e}")

    pdf_file.close()

In [45]:
extract_images_from_pdf(Path_File, output_path)

Error on page 1: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 1: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 1: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 3: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 4: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 6: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 7: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 8: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 8: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 9: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 9: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 10: 'EncodedStreamObject' object has no attribute 'get_images'
Error on page 10: 'EncodedStreamObject' object has no attribute 'get_images'
Error on p

In [29]:
reader = PdfReader(Path_File)

In [30]:
reader.pages[0].images

In [6]:
from unstructured.partition.pdf import partition_pdf

c:\Users\pablosal\AppData\Local\anaconda3\envs\document-intelligence\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(
    filename=Path_File,
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=output_path,
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pablosal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pablosal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
yolox_l0.05.onnx: 100%|██████████| 217M/217M [00:04<00:00, 45.2MB/s] 


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?